In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, RepeatVector, TimeDistributed, Concatenate, Bidirectional, Activation, Multiply
import tensorflow as tf

In [2]:
num_samples = 1000
sequence_length = 50

# Create synthetic data (example)
df = pd.DataFrame({
    'packet_size': np.random.normal(loc=500, scale=100, size=num_samples),
    'connection_duration': np.random.normal(loc=2, scale=0.5, size=num_samples),
    'packet_inter_arrival_time': np.random.normal(loc=0.02, scale=0.005, size=num_samples),
    'SYN_packets': np.random.randint(0, 10, size=num_samples),
    'packet_size_connection_ratio': np.random.normal(loc=250, scale=50, size=num_samples),
    'SYN_packet_ratio': np.random.uniform(0, 1, size=num_samples)
})

In [4]:
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df)

# Prepare sequences
X_train_data = []
y_train_data = []

for i in range(num_samples - sequence_length):
    X_train_data.append(scaled_features[i:i + sequence_length])
    # For target, take the next value after the sequence
    y_train_data.append(scaled_features[i + sequence_length, [0, 1]])  # Changed this line

X_train_data = np.array(X_train_data)
y_train_data = np.array(y_train_data)

print("Shape of X_train_data:", X_train_data.shape)  # Should be (950, 50, 6)
print("Shape of y_train_data:", y_train_data.shape)  # Should be (950, 2)


Shape of X_train_data: (950, 50, 6)
Shape of y_train_data: (950, 2)


In [6]:
def attention_layer(inputs):
    """
    Applies an attention mechanism on the input sequence.
    """
    attention = Dense(1, activation="tanh")(inputs)
    attention = Flatten()(attention)
    attention = Activation("softmax")(attention)
    attention = RepeatVector(inputs.shape[-1])(attention)
    attention = Permute([2, 1])(attention)
    
    output_attention = Multiply()([inputs, attention])
    return output_attention

In [7]:
input_layer = Input(shape=(sequence_length, 6))

# LSTM branch
lstm_branch = LSTM(64, activation='relu', return_sequences=True)(input_layer)
lstm_branch = LSTM(32, activation='relu', return_sequences=True)(lstm_branch)

# Bidirectional LSTM branch
bidirectional_branch = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(input_layer)
bidirectional_branch = Bidirectional(LSTM(32, activation='relu', return_sequences=True))(bidirectional_branch)

# GRU branch
gru_branch = GRU(64, activation='relu', return_sequences=True)(input_layer)
gru_branch = GRU(32, activation='relu', return_sequences=True)(gru_branch)

# Concatenate all branches
combined = Concatenate()([lstm_branch, bidirectional_branch, gru_branch])

# Apply attention mechanism
attention_output = attention_layer(combined)

In [8]:
# Flatten and dense layers for final prediction
x = Flatten()(attention_output)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(2)(x)  # Predicting next packet_size and connection_duration

In [ ]:
# Build and compile the model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Print model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 50, 6)]              0         []                            
                                                                                                  
 lstm (LSTM)                 (None, 50, 64)               18176     ['input_1[0][0]']             
                                                                                                  
 bidirectional (Bidirection  (None, 50, 128)              36352     ['input_1[0][0]']             
 al)                                                                                              
                                                                                                  
 gru (GRU)                   (None, 50, 64)               13824     ['input_1[0][0]']         

In [11]:
history = model.fit(
    X_train_data, 
    y_train_data,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/50
24/24 [==============================] - 2s 100ms/step - loss: 0.0236 - mae: 0.1215 - val_loss: 0.0232 - val_mae: 0.1199
Epoch 2/50
24/24 [==============================] - 2s 104ms/step - loss: 0.0233 - mae: 0.1200 - val_loss: 0.0231 - val_mae: 0.1184
Epoch 3/50
24/24 [==============================] - 3s 115ms/step - loss: 0.0236 - mae: 0.1212 - val_loss: 0.0238 - val_mae: 0.1203
Epoch 4/50
24/24 [==============================] - 3s 114ms/step - loss: 0.0236 - mae: 0.1209 - val_loss: 0.0237 - val_mae: 0.1204
Epoch 5/50
24/24 [==============================] - 3s 133ms/step - loss: 0.0232 - mae: 0.1202 - val_loss: 0.0227 - val_mae: 0.1179
Epoch 6/50
24/24 [==============================] - 3s 126ms/step - loss: 0.0232 - mae: 0.1205 - val_loss: 0.0243 - val_mae: 0.1218
Epoch 7/50
24/24 [==============================] - 3s 128ms/step - loss: 0.0231 - mae: 0.1200 - val_loss: 0.0228 - val_mae: 0.1180
Epoch 8/50
24/24 [==============================] - 3s 125ms/step - loss: 0.

In [12]:
model.save('model4.h5')

C:\Users\ANANT TIWARI\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
